In [5]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

import keras
import keras_tuner
from k_fold_cv.k_fold_generator import KFoldController
from utils.dataset_loader import dataset_loader
import keras_tuner
from k_fold_cv.k_fold_generator import KFoldController
from utils.dataset_loader import dataset_loader
import keras_tuner
from torch.utils.data import DataLoader
from models.naive_dnn_gen.naive_dnn import NaiveDNNTunableModelFamily
from models.structure.tunable_model_family_hypermodel import TunableModelFamilyHypermodel
from torch.utils.data import DataLoader

In [6]:

from utils.my_tuner import HyperBandSaveLess

hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((128, 128), is_grayscale=False)

dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, batch_size=16, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=16, shuffle=True)

model = NaiveDNNTunableModelFamily()
hypermodel = TunableModelFamilyHypermodel((3, 128, 128), model)

# Adam is too heavy
hyperparameters.Fixed("optimizer", "SGD")
hyperparameters.Fixed("layers", 2)

tuner = HyperBandSaveLess(
    hypermodel,
    hyperparameters=hyperparameters,
    objective='val_loss',
    tune_new_entries=True,
    overwrite=True,
    directory="dnn-search",
    max_trials=15,
    project_name="standard_image"
)

[(256, None), (256, None)]


In [7]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
optimizer (Fixed)
{'conditions': [], 'value': 'SGD'}
layers (Fixed)
{'conditions': [], 'value': 2}
units_0 (Int)
{'default': 256, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 2, 'sampling': 'log'}
dropout_0 (Boolean)
{'default': False, 'conditions': []}
units_1 (Int)
{'default': 256, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 2, 'sampling': 'log'}


In [8]:
tuner.search(train_dataloader, epochs=2, validation_data=validation_dataloader)

Trial 1 Complete [00h 01m 24s]
val_loss: 0.6720849871635437

Best val_loss So Far: 0.6720849871635437
Total elapsed time: 00h 01m 24s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
SGD               |SGD               |optimizer
2                 |2                 |layers
32                |32                |units_0
True              |True              |dropout_0
64                |32                |units_1

[(32, 0.3), (64, None)]
Epoch 1/2
 54/237 ━━━━━━━━━━━━━━━━━━━━ 27s 148ms/step - accuracy: 0.5489 - loss: 0.7532

KeyboardInterrupt: 

In [ ]:
## TODO MA PERCHÈ NON VA? SEMBRA ASSOLUTAMENTE NORMALE (256)(1024) ma loss è a 7. Potrebbe essere che ho troppi neuroni al secondo layer?
# Sembra di no. Fa schifo per qualche motivo non noto. Indaga poi

# PAre funzionare come previsto From the observations better tailor the search space

tuner.results_summary()

Results summary
Results in dnn-search/standard_image
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 08 summary
Hyperparameters:
optimizer: adam
layers: 2
units_0: 64
dropout_0: True
units_1: 32
Score: 0.606936514377594

Trial 14 summary
Hyperparameters:
optimizer: adam
layers: 2
units_0: 32
dropout_0: True
units_1: 32
Score: 0.6195433735847473

Trial 03 summary
Hyperparameters:
optimizer: adam
layers: 2
units_0: 32
dropout_0: False
units_1: 1024
Score: 0.6893175840377808

Trial 07 summary
Hyperparameters:
optimizer: adam
layers: 2
units_0: 32
dropout_0: True
units_1: 256
Score: 0.6893264651298523

Trial 10 summary
Hyperparameters:
optimizer: adam
layers: 2
units_0: 32
dropout_0: True
units_1: 128
Score: 0.6893460154533386

Trial 04 summary
Hyperparameters:
optimizer: adam
layers: 2
units_0: 512
dropout_0: False
units_1: 32
Score: 7.353879451751709

Trial 00 summary
Hyperparameters:
optimizer: adam
layers: 2
units_0: 256
dropout_0: True
units_1: 1024
Score: 7.353880882263184

Trial 06 summary
Hyperparameters:
optimizer: adam
layers: 2
units_0: 1024
dropout_0: True
units_1: 1024
Score: 7.353880882263184

Trial 02 summary
Hyperparameters:
optimizer: adam
layers: 2
units_0: 256
dropout_0: False
units_1: 512
Score: 7.353881359100342

Trial 13 summary
Hyperparameters:
optimizer: adam
layers: 2
units_0: 128
dropout_0: True
units_1: 128
Score: 7.3538818359375

In [ ]:

from utils.my_tuner import HyperBandSaveLess

hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((128, 128), is_grayscale=False)

dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, batch_size=16, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=16, shuffle=True)

model = NaiveDNNTunableModelFamily()
model.hidden_layers = [(32, None), (1024, None)]

model_instance =model.make_model((3,128,128))
model.compile_model(model_instance, 'adam')
model_instance.summary()

In [ ]:
model_instance.fit(train_dataloader, epochs=12, validation_data=validation_dataloader)

In [ ]:
# Va proprio forte questa configurazione pare. Tienila e studia i possiibli parametri
input_layer = keras.Input(shape=(3,128,128), batch_size=16)
x = keras.layers.Flatten(data_format="channels_first")(input_layer)
x = keras.layers.Dense(units=3500, activation="relu")(x)
x = keras.layers.Dense(units=1350, activation="relu")(x)

output_layer = keras.layers.Dense(units=1, activation='sigmoid')(x)

model = keras.Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
train, test = dataset_loader((128, 128), is_grayscale=False)

dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, batch_size=16, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=16, shuffle=True)

model.fit(train_dataloader, epochs=12, validation_data=validation_dataloader)